In [6]:
import mne
import warnings  # Hide all warnings here
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import mne
import warnings  # Hide all warnings here


In [7]:
def load_pandas_data(filename):
    # When exporting data as a NumPy array or Pandas DataFrame, be sure to properly account for the unit of representation in your subsequent analyses.
    # with preload=True, contents are read into RAM

    mne.set_log_level('WARNING')
    warnings.simplefilter("ignore")

    patient = mne.io.read_raw(filename,preload=True)
    return patient.to_data_frame()


def get_seizure(seizure_set_df, start, end):
    """ return index locations for a given seizure set """

    start = np.where(seizure_set_df['time'] == start)
    end = np.where(seizure_set_df['time'] == end)
    seizure_set_df.loc[start[0][0]:end[0][0],'seizure = 1'] = 1
    return seizure_set_df


def prepare_df_for_analysis(preseizure_set, seizure_set):
    """ prepare the dataframe for the analysis """

    # seizure_set['time'] = seizure_set['time'] + 3600.000001
    p1 = pd.concat([preseizure_set, seizure_set],axis=0,ignore_index=True)
    p1.set_index('time',inplace=True)

    # left hemisphere location totals
    p1['outer_left_hemi_sum'] = p1[['FP1-F7','F7-T7','T7-P7','P7-O1']].sum(axis=1)
    p1['inner_left_hemi_sum'] = p1[['FP1-F3','F3-C3','C3-P3','P3-O1']].sum(axis=1)

    # right hemisphere location totals
    p1['outter_right_hemi_sum'] = p1[['FP2-F8','F8-T8','T8-P8-0','P8-O2']].sum(axis=1)
    p1['inner_right_hemi_sum'] = p1[['FP2-F4','F4-C4','C4-P4','P4-O2']].sum(axis=1)

    # center totals
    p1['center_line'] = p1[['FZ-CZ','CZ-PZ']].sum(axis=1)

    # left temple to rear totals
    p1['left_temple_to_left_rear'] = p1[['FZ-CZ','CZ-PZ']].sum(axis=1)

    # temple to temple totals
    p1['temple_to_temple'] = p1[['FT9-FT10']].sum(axis=1)

    # left temple to rear totals
    p1['right_temple_to_right_rear'] = p1[['FT10-T8','T8-P8-1']].sum(axis=1)

    # reorder columns for better visual early analysis
    new_cols = ['seizure = 1',
                'FP2-F4','FP1-F7','F7-T7','T7-P7','P7-O1','outer_left_hemi_sum',
                'FP1-F3','F3-C3','C3-P3','P3-O1','inner_left_hemi_sum',
                'FP2-F8','F8-T8','T8-P8-0','P8-O2','outter_right_hemi_sum',
                'FP2-F4','F4-C4','C4-P4','P4-O2','inner_right_hemi_sum',
                'FZ-CZ','CZ-PZ','center_line',
                'P7-T7','T7-FT9','left_temple_to_left_rear',
                'FT9-FT10','temple_to_temple',
                'FT10-T8','T8-P8-1','right_temple_to_right_rear']

    p1 = p1.reindex(columns=new_cols)
    return p1


def is_df_clean(df):
    """ Check if a dataframe is clean. """
    # is_na = df.count()
    for i in range(df.count().all().tolist()):
        if not i:
            return f"There are null values in {df.columns[i]}: Note: this is expected if column is 'seizures = 1' as most of the values are zero, but 1 where the seizure exist."
    # infinite = np.isinf(df).sum()
    for j in  np.isinf(df).sum().any().to_list():
        if not j:
            return f'There are infinite values in {df.columns[j]}'



In [8]:
data_set_site = 'https://physionet.org/content/chbmit/1.0.0/ '
view_data_set ="https://physionet.org/lightwave/?db=chbmit/1.0.0"
quick_access_mne_raw ="https://mne.tools/dev/generated/mne.io.Raw.html"

mne.set_log_level('WARNING')
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore",category=DeprecationWarning)

raw_p1_02_nonseizure = '/Users/jshensley/Desktop/springboard/DS_Method_and_Capstones/capstone_2_2/capstone_2/a_cookiecutter_for_capstone_2/{{ cookiecutter.seizure_prediction_model }}/pycharm_files/Capstone2_dataset/patient_1/intial_EEG/chb01_02_nonseizure_set_just_before_seizure_set.edf'

raw_p1_02_seizure_set = '/Users/jshensley/Desktop/springboard/DS_Method_and_Capstones/capstone_2_2/capstone_2/a_cookiecutter_for_capstone_2/{{ cookiecutter.seizure_prediction_model }}/pycharm_files/Capstone2_dataset/patient_1/intial_EEG/chb01_03_aa_seizure_set.edf'

p1_preseizure = load_pandas_data(raw_p1_02_nonseizure)
p1_preseizure['seizure = 1'] = 0

p1_seizure_set = load_pandas_data(raw_p1_02_seizure_set)
p1_seizure_set['seizure = 1'] = 0


p1_seizures = get_seizure(p1_seizure_set, 2996, 3036)
p1_df = prepare_df_for_analysis(p1_preseizure, p1_seizure_set)
p1_df_is_clean = is_df_clean(p1_df)
print(p1_df_is_clean)

p1_df.info()
print(f'\n\n\n')
p1_describe = p1_df.describe()
print(p1_describe)











There are null values in seizure = 1: Note: this is expected if column is 'seizures = 1' as most of the values are zero, but 1 where the seizure exist.
<class 'pandas.core.frame.DataFrame'>
Float64Index: 1843200 entries, 0.0 to 3599.99609375
Data columns (total 33 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   seizure = 1                 int64  
 1   FP2-F4                      float64
 2   FP1-F7                      float64
 3   F7-T7                       float64
 4   T7-P7                       float64
 5   P7-O1                       float64
 6   outer_left_hemi_sum         float64
 7   FP1-F3                      float64
 8   F3-C3                       float64
 9   C3-P3                       float64
 10  P3-O1                       float64
 11  inner_left_hemi_sum         float64
 12  FP2-F8                      float64
 13  F8-T8                       float64
 14  T8-P8-0                     float64
 15  P8-O2                  